# Data Preparation for 2025 Milk Collection Route Optimization

Determine which depots use "Shipping" vs "Truck" collection based on:
- Single driver, max 12 hours (720 min) per day
- Distance/time constraints

**Input:** Distance/time matrix + 2025 forecast data
**Output:** Strategy table + filtered forecast for truck routes

## 1. Setup

In [24]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Paths
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent
DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

## 2. Load Data

In [25]:
# Load distance/time matrix
distance_df = pd.read_excel(DATA_RAW / "Optional_Driving_Distance_and_Time.xlsx", sheet_name=0)

# Load forecast data
forecast_df = pd.read_csv(DATA_PROCESSED / "forecast_2025.csv")
forecast_df['Date_2025'] = pd.to_datetime(forecast_df['Date_2025'])

print(f"Loaded {len(distance_df):,} distance/time records")
print(f"Loaded {len(forecast_df):,} forecast records for {forecast_df['Depo'].nunique()} depots")

Loaded 1,332 distance/time records
Loaded 4,262 forecast records for 37 depots


## 3. Identify Hub Location

Find the Milk Bank (hub) from the distance matrix.

In [26]:
# Identify columns
FROM_COL = 'Depot 1 Designation'
TO_COL = 'Depot 2 Designation'
TIME_COL = 'Driving Time (minutes)'
DISTANCE_COL = 'Driving Distance (miles)'

# Find hub (Milk Bank)
all_locations = set(distance_df[FROM_COL].unique()) | set(distance_df[TO_COL].unique())
hub_name = next((loc for loc in all_locations if 'milk bank' in str(loc).lower()), None)

if not hub_name:
    # Fallback: most common origin
    hub_name = distance_df[FROM_COL].value_counts().index[0]

print(f"Hub identified: {hub_name}")

Hub identified: Oklahoma Mothers’ Milk Bank


## 4. Calculate Round Trip Times

For each depot, calculate: Hub → Depot + Depot → Hub

In [27]:
# Service time assumptions
PICKUP_TIME = 15  # minutes at depot
DROPOFF_TIME = 15  # minutes at hub
MAX_SAFE_MINUTES = 660  # 11 hours (leaves 60 min buffer)

# Extract hub routes
hub_to_depot = distance_df[distance_df[FROM_COL] == hub_name].copy()
depot_to_hub = distance_df[distance_df[TO_COL] == hub_name].copy()

# Calculate round trip times
def calc_round_trip(depot_name):
    if depot_name == hub_name:
        return 0.0, 0.0, 0.0
    
    hub_to = hub_to_depot[hub_to_depot[TO_COL] == depot_name][TIME_COL]
    depot_to = depot_to_hub[depot_to_hub[FROM_COL] == depot_name][TIME_COL]
    
    hub_to_time = hub_to.values[0] if len(hub_to) > 0 else np.nan
    depot_to_time = depot_to.values[0] if len(depot_to) > 0 else np.nan
    round_trip = hub_to_time + depot_to_time if not (np.isnan(hub_to_time) or np.isnan(depot_to_time)) else np.nan
    
    return hub_to_time, depot_to_time, round_trip

# Create depot constraints table
all_depots = sorted(forecast_df['Depo'].unique())
results = []
for depot in all_depots:
    hub_to_time, depot_to_time, round_trip = calc_round_trip(depot)
    results.append({
        'Depot': depot,
        'Hub_to_Depot_Time': hub_to_time,
        'Depot_to_Hub_Time': depot_to_time,
        'Round_Trip_Time': round_trip
    })

depot_constraints = pd.DataFrame(results)
print(f"Calculated round trip times for {len(depot_constraints)} depots")

Calculated round trip times for 37 depots


## 5. Calculate 2025 Total Volumes

Sum forecast volumes by depot for ROI analysis.

In [28]:
# Calculate total volume per depot
depot_volumes = forecast_df.groupby('Depo')['Volume_2025'].sum().reset_index()
depot_volumes.columns = ['Depot', 'Total_Volume_2025']

# Merge into constraints
depot_constraints = depot_constraints.merge(depot_volumes, on='Depot', how='left')
depot_constraints['Total_Volume_2025'] = depot_constraints['Total_Volume_2025'].fillna(0)

print(f"Total volume across all depots: {depot_constraints['Total_Volume_2025'].sum():,.0f} oz")

Total volume across all depots: 486,834 oz


## 6. Apply Shipping vs Truck Rules

**Rule 1: Traffic Buffer** - If total task time > 660 min → Shipping
**Rule 2: ROI Efficiency** - If round trip > 300 min AND volume < 2000 oz → Shipping

In [29]:
# Calculate total task time
depot_constraints['Total_Task_Time'] = depot_constraints['Round_Trip_Time'] + PICKUP_TIME + DROPOFF_TIME

# Initialize all as Truck Candidate
depot_constraints['Strategy'] = 'Truck Candidate'
depot_constraints['Shipping_Reason'] = ''

# Rule 1: Traffic Buffer (Hard Limit)
traffic_mask = depot_constraints['Total_Task_Time'] > MAX_SAFE_MINUTES
depot_constraints.loc[traffic_mask, 'Strategy'] = 'Shipping'
depot_constraints.loc[traffic_mask, 'Shipping_Reason'] = 'Risk of exceeding 12h shift due to traffic'

# Rule 2: ROI Efficiency (Economic Limit)
far_threshold = 300  # 5 hours
low_yield_threshold = 2000  # oz
roi_mask = (
    (depot_constraints['Round_Trip_Time'] > far_threshold) &
    (depot_constraints['Total_Volume_2025'] < low_yield_threshold) &
    (depot_constraints['Strategy'] == 'Truck Candidate')
)
depot_constraints.loc[roi_mask, 'Strategy'] = 'Shipping'
depot_constraints.loc[roi_mask, 'Shipping_Reason'] = 'Low annual yield does not justify long-haul truck routes'

# Summary
truck_count = (depot_constraints['Strategy'] == 'Truck Candidate').sum()
shipping_count = (depot_constraints['Strategy'] == 'Shipping').sum()
print(f"Strategy assignment:")
print(f"  Truck Candidate: {truck_count} depots")
print(f"  Shipping: {shipping_count} depots")

# Show shipping depots
shipping_depots = depot_constraints[depot_constraints['Strategy'] == 'Shipping']
if len(shipping_depots) > 0:
    print(f"\nShipping depots:")
    for _, row in shipping_depots.iterrows():
        print(f"  - {row['Depot']}: {row['Shipping_Reason']}")

Strategy assignment:
  Truck Candidate: 36 depots
  Shipping: 1 depots

Shipping depots:
  - Andrew County: Risk of exceeding 12h shift due to traffic


## 7. Save Outputs

In [30]:
# Prepare strategy table
strategy_output = depot_constraints[[
    'Depot', 'Strategy', 'Round_Trip_Time', 'Hub_to_Depot_Time',
    'Depot_to_Hub_Time', 'Total_Task_Time', 'Total_Volume_2025', 'Shipping_Reason'
]].copy()
strategy_output = strategy_output.rename(columns={'Depot': 'Depo'})
strategy_output['Shipping_Reason'] = strategy_output['Shipping_Reason'].fillna('')

# Merge strategy into forecast
forecast_with_strategy = forecast_df.merge(
    strategy_output[['Depo', 'Strategy']], on='Depo', how='left'
)
forecast_with_strategy['Strategy'].fillna('Shipping', inplace=True)

# Filter to truck-only
forecast_truck_only = forecast_with_strategy[
    forecast_with_strategy['Strategy'] == 'Truck Candidate'
].copy()

# Save files
strategy_output.to_csv(DATA_PROCESSED / "depot_service_strategy.csv", index=False)
forecast_truck_only.to_csv(DATA_PROCESSED / "forecast_2025_truck_only.csv", index=False)

print(f"Saved files:")
print(f"  - depot_service_strategy.csv ({len(strategy_output)} depots)")
print(f"  - forecast_2025_truck_only.csv ({len(forecast_truck_only):,} rows)")
print(f"\n✓ Done! Ready for optimization model.")

Saved files:
  - depot_service_strategy.csv (37 depots)
  - forecast_2025_truck_only.csv (4,197 rows)

✓ Done! Ready for optimization model.
